In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "../src"))

In [2]:
import logging
import jax
import jax.numpy as np
import numpy as onp
import skbio.stats.composition as cmp
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import statsmodels.api as sm
from datetime import datetime

from simulate_data_fct import sim_IV_ilr_linear, sim_IV_ilr_nonlinear
from plot_fct import update_layout, img_path, update_layout_px, colours, plot_ilr_X_vs_Y
from run_methods_all import run_methods_confidence_interval
from plot_fct import plot_mse_results, plot_beta_results

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# set up logging file
logging.getLogger().setLevel(logging.INFO)

In [5]:
key = jax.random.PRNGKey(253)

# Linear Setup

## Data Generation

In [17]:
n = 1000
p = 3
V = cmp._gram_schmidt_basis(p)
instrument_strength = "strong"
savepath = os.path.join(img_path, "NonLinearSetUp")
num_inst = 2
c_X = np.array([2, 2])  # confounder multiplication to X
alpha0 = np.array([1, 1])  # INSTRUMENT intercept
alphaT = np.array([[4, 1],  # INSTRUMENT INTERCEPT FIrst Component of Microbiome
                    [-1, 3]])  # INSTRUMENT INTERCEPT Second Component of Microbiome
beta0 = 5
betaT = np.array([6, 2])
mu_c = -1
c_Y = 4

In [7]:
confounder, Z_sim, X_sim, Y_sim, X_star, Y_star = sim_IV_ilr_nonlinear(
            key,
            n=n,
            p=p,
            num_inst=num_inst,
            mu_c=mu_c,
            c_X=c_X,
            alpha0=alpha0,
            alphaT=alphaT,
            c_Y=c_Y,
            beta0=beta0,
            betaT=betaT,
            num_star=500)
X_sim_ilr = cmp.ilr(X_sim)
X_star_ilr = cmp.ilr(X_star)

## Data Visualization

In [18]:
fig = px.bar(pd.DataFrame(X_sim[:100, :], columns=["Microbiota 1", "Mictobiota 2", "Microbiota 3"]))
fig = update_layout_px(fig)
fig.update_xaxes(title="Sample number")
fig.update_yaxes(title="Percentage of microbiota in sample")
fig.write_image(os.path.join(savepath,
                             "Barplot_Sample_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

In [19]:
# Compute F-Statistics 
ZZ_sim = onp.array(sm.add_constant(Z_sim))
alphahat = np.linalg.inv(ZZ_sim.T@ZZ_sim)@ZZ_sim.T@X_sim_ilr
MSM = np.sum(((ZZ_sim@alphahat - np.mean(X_sim_ilr, axis=0))**2), axis=0)/(num_inst)
MSE = np.sum(((ZZ_sim@alphahat - X_sim_ilr)**2), axis=0)/(n-num_inst)
F = MSM / MSE
F

DeviceArray([148.2662 , 100.69248], dtype=float32)

In [20]:
df = pd.DataFrame(X_sim, columns=["Microbiota 1", "Microbiota 2", "Microbiota 3"])
fig = px.scatter_ternary(df, a='Microbiota 1', 
                         b='Microbiota 2', 
                         c='Microbiota 3',
                         color=Z_sim[:, 0].squeeze())

fig = update_layout_px(fig)
#ig.update_layout(gridcolor="gray")
fig.write_image(os.path.join(savepath,
                             "Ternaryplot_SampleZ1_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

In [21]:
fig = px.scatter_ternary(df, a='Microbiota 1', 
                         b='Microbiota 2', 
                         c='Microbiota 3',
                         color=Z_sim[:, 1].squeeze())

fig = update_layout_px(fig)
fig.write_image(os.path.join(savepath,
                             "Ternaryplot_SampleZ2_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

In [22]:
fig = plot_ilr_X_vs_Y(X_sim_ilr, Y_sim, Y_star)

fig.write_image(os.path.join(savepath,
                             "Data_vs_trueeffect_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

# Diversity Approach

In [13]:
from plot_fct import plot_diversity_methods
from helper_fct import diversity
from run_methods_all import run_diversity_estimation_methods
div_shannon = diversity(X_sim, "shannon")
div_simpson = diversity(X_sim, "simpson")

In [24]:
# Shannon Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_shannon, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

fig = plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(savepath,
                             "ShannonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     61.99
Date:                 Di, 01 Jun 2021   Prob (F-statistic):           4.22e-26
Time:                        16:53:43   Log-Likelihood:                -1360.3
No. Observations:                1000   AIC:                             2727.
Df Residuals:                     997   BIC:                             2741.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5494      0.079      6.920      0.0

In [25]:
# Simpson Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_simpson, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(savepath,
                             "SimpsonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     57.49
Date:                 Di, 01 Jun 2021   Prob (F-statistic):           2.35e-24
Time:                        16:53:49   Log-Likelihood:                -1364.4
No. Observations:                1000   AIC:                             2735.
Df Residuals:                     997   BIC:                             2749.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5329      0.080      6.685      0.0

# Confidence Intervals for higher dimensional methods

In [19]:
from run_methods_all import run_methods_all

In [16]:
today = datetime.now()
num_iteration = 20

path = os.getcwd()
mypath = os.path.join(path, "temp",
                      today.strftime("%Y_%m_%d_%H_%M") + "_NONLINEAR_ConfidenceInterval_"+str(instrument_strength)
                      +"_"+ str(num_iteration)+"_Microbiota_" +str(p))

if not os.path.isdir(mypath):
    os.makedirs(mypath)


# create logging file with relevant information
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    filename=os.path.join(os.getcwd(), mypath, "configurations.log"),
                    filemode='w'
                    )

logging.info("n="+str(n)+", p="+str(p)+", instrument strength: "+instrument_strength)
logging.info("Number of runs: "+str(num_iteration))
logging.info("First Stage Specification: ")
logging.info("Number of Instruments: "+str(num_inst))
logging.info("Instrument Strength by F-Test: " +str(F))
logging.info("alpha0="+str(alpha0)+", alphaT="+str(alphaT)+", confounder multiplicator c_X="+str(c_X))
logging.info("Second Stage Specification: ")
logging.info("beta0="+str(beta0)+", betaT="+str(betaT)+", confounder multiplicator c_Y="+str(c_Y))
logging.info("Confounder mean mu_c="+str(mu_c))


df_beta, df_mse, mse_large_confidence= run_methods_confidence_interval(
    key, num_iteration, 
    n, p, num_inst, mu_c, c_X, alpha0, alphaT, c_Y, beta0, betaT, 
    is_nonlinear=True,
    num_star=200)

**************************************************************************************************
*****************************************We are at 0 of 20***********************************************
**************************************************************************************************


/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/dirichlet/dirichlet.py:265: RuntimeWarning:

Mean of empty slice.

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in true_divide



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [3.03 2.33]
Estimated Beta: [ 3.09 -1.19 -1.9 ]
Error: 27.7

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [ 3.09 -1.19 -1.9 ]
Error: 27.7

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Dirichlet + Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Beta DirichLetLogContrast: 1.39[ 6.672 -4.526 -2.146]
True Beta: [6 2]
Estimated Beta: [7.92 2.63]
Error: 351.53

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - ILR + Log Contrast >>

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/dirichlet/dirichlet.py:265: RuntimeWarning:

Mean of empty slice.

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in true_divide



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [2.83 2.14]
Estimated Beta: [ 2.87 -1.13 -1.75]
Error: 20.95

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [ 2.87 -1.13 -1.75]
Error: 20.95

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Dirichlet + Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Beta DirichLetLogContrast: 1.962[ 5.371 -3.957 -1.415]
True Beta: [6 2]
Estimated Beta: [6.6  1.73]
Error: 183.32

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - ILR + Log Contrast

...

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [3.03 2.17]
Estimated Beta: [ 3.03 -1.26 -1.77]
Error: 27.89

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [ 3.03 -1.26 -1.78]
Error: 27.9

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Dirichlet + Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Beta DirichLetLogContrast: 2.344[ 4.131 -3.963 -0.168]
True Beta: [6 2]
Estimated Beta: [5.72 0.21]
Error: 95.61

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - ILR + Log Contrast >

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/dirichlet/dirichlet.py:265: RuntimeWarning:

Mean of empty slice.

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in true_divide



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [2.91 2.41]
Estimated Beta: [ 3.04 -1.08 -1.96]
Error: 27.94

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [ 3.04 -1.08 -1.96]
Error: 27.94

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Dirichlet + Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Beta DirichLetLogContrast: 1.633[ 6.715 -4.506 -2.209]
True Beta: [6 2]
Estimated Beta: [7.93 2.71]
Error: 333.04

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - ILR + Log Contrast

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/dirichlet/dirichlet.py:265: RuntimeWarning:

Mean of empty slice.

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in true_divide



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [2.98 2.21]
Estimated Beta: [ 3.01 -1.2  -1.81]
Error: 38.15

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [6 2]
Estimated Beta: [ 3.01 -1.2  -1.81]
Error: 38.14

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - Dirichlet + Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Beta DirichLetLogContrast: 2.107[ 4.893 -4.21  -0.683]
True Beta: [6 2]
Estimated Beta: [6.44 0.84]
Error: 162.41

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2SLS - ILR + Log Contrast

In [26]:
import pickle
results = {
    "df_mse": df_mse,
    "df_beta": df_beta,
    "mse_large": mse_large_confidence
}

with open(os.path.join(savepath, "results_"+str(p)+str(instrument_strength)+".pickle"), "wb") as f:
    pickle.dump(results, f)
    f.close()


# Mean Squarred Error Plot

In [27]:
filter_list = ["ALR+LC", "ILR+ILR", "ONLY Second ILR", "ONLY Second LC", "DIR+LC", "ILR+LC"]

fig = plot_mse_results(df_mse, filter_list)
fig.update_yaxes(range=(0, 100))
fig.write_image(os.path.join(savepath,
                             "MSEFull_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))
fig.show()

# Beta Plot

In [28]:
fig = plot_beta_results(df_beta, V.T@betaT, filter_list)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_yaxes(range=(-20, 40))
fig.write_image(os.path.join(savepath,
                             "BetaFull_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()
